<a href="https://colab.research.google.com/github/ahmad811/nlp/blob/main/commands_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/cmd2id.pickle', 'rb') as f:
    cmd2idMap = pickle.load(f)
with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/id2cmd.pickle', 'rb') as f:
    id2cmdMap = pickle.load(f)

with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/book_np.pickle', 'rb') as f:
    corpus = pickle.load(f)

unique_cmds = cmd2idMap.keys()

corpus_len = len(corpus)
vocab_len = len(unique_cmds)

print(corpus_len)
print(vocab_len)


211292812
809


In [5]:
seq_len = 10

X=[]
y=[]

first_chunk=1000_000

for i in range(seq_len+1, first_chunk):#corpus_len):
    seq = corpus[i-seq_len-1:i]
    X.append(seq[:-1]) #all but last one
    y.append(seq[-1]) #last one


In [6]:
#first_chunk=1000_000
#X_1m = np.array(X[:first_chunk], dtype='int16')
X = np.array(X, dtype='int16')
#y_1m = y[:first_chunk]

y = to_categorical(y, num_classes=vocab_len, dtype=bool)

print(X.shape)
print(y.shape)

(999989, 10)
(999989, 809)


In [7]:
#Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_len, output_dim=512, input_length=seq_len))
model.add(LSTM(2048, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_len, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 512)           414208    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 2048)          20979712  
_________________________________________________________________
dropout (Dropout)            (None, 10, 2048)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              12587008  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 809)               8

In [8]:
#Callbacks
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextcommand.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')



In [ ]:

#Compile and Fit
model.compile (loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
model.fit(X, y, batch_size=512, epochs=100, callbacks=[checkpoint])


Epoch 1/100
1954/1954 [==============================] - 1064s 539ms/step - loss: 2.3062 - accuracy: 0.4646

Epoch 00001: loss improved from inf to 2.30619, saving model to nextcommand.h5
Epoch 2/100
1954/1954 [==============================] - 1052s 538ms/step - loss: 1.9962 - accuracy: 0.5098

Epoch 00002: loss improved from 2.30619 to 1.99618, saving model to nextcommand.h5
Epoch 3/100
1954/1954 [==============================] - 1059s 542ms/step - loss: 1.9060 - accuracy: 0.5225

Epoch 00003: loss improved from 1.99618 to 1.90595, saving model to nextcommand.h5
Epoch 4/100
1954/1954 [==============================] - 1059s 542ms/step - loss: 1.8250 - accuracy: 0.5346

Epoch 00004: loss improved from 1.90595 to 1.82501, saving model to nextcommand.h5
Epoch 5/100
1954/1954 [==============================] - 1057s 541ms/step - loss: 1.7300 - accuracy: 0.5498

Epoch 00005: loss improved from 1.82501 to 1.72998, saving model to nextcommand.h5
Epoch 6/100
1954/1954 [=====================

In [ ]:
model.save('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/model')
model.save_weights('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/model.weights')

# New Section